In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import json

@udf("map<string, string>")
def parse(s):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        pass 

TOPIC = "testhub2"
BOOTSTRAP_SERVERS = "prmongoeh.servicebus.windows.net:9093"
EH_SASL = "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"Endpoint=sb://prmongoeh.servicebus.windows.net/;SharedAccessKeyName=t2;SharedAccessKey=mgS8U4GgXdsp8HXD00FDQH3PM6me9UnYI0fO0/LrD7I=\";"

schema = StructType() \
        .add("count", StringType()) 

df = spark.readStream.format("kafka")\
.option("subscribe", TOPIC)\
.option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS)\
.option("kafka.sasl.mechanism", "PLAIN")\
.option("kafka.security.protocol", "SASL_SSL")\
.option("kafka.sasl.jaas.config", EH_SASL)\
.option("kafka.request.timeout.ms", "60000")\
.option("kafka.session.timeout.ms", "60000")\
.option("startingOffsets", """{"testhub2":{"0":1}}""")\
.option("kafka.group.id","default")\
.option("failOnDataLoss", "false")\
.load()\
.select(explode(parse("value")).alias("st","count")).select("count")
#.selectExpr("CAST(value AS STRING) as data")


In [0]:
display(df, streamName="debug_stream")


count
47
48
49
50
51
52
53
54
55
56


In [0]:
df.writeStream.format("mongodb") \
  .option("spark.mongodb.connection.uri", "mongodb://prcosmosdbtest:jLNdr4yFLFj5MrlZlcjHOu3auaYjg0zMwMiTJY8O0OMqS3QyxCzGsuQrAQUr0vZFfPD5Vc5zPEggzIr3gZKcoA==@prcosmosdbtest.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@prcosmosdbtest@") \
  .option("spark.mongodb.database", "db") \
  .option("spark.mongodb.collection", "col") \
  .option("checkpointLocation", "dbfs:/tmp/check1129") \
  .outputMode("append").start()


Out[77]: <pyspark.sql.streaming.StreamingQuery at 0x7fe0e9d40250>